In [1]:
# vim: expandtab:ts=4:sw=4
import argparse

import os
import cv2
import math
import shutil
import numpy as np
import pandas as pd
import tensorflow as tf

from math import factorial
from deep_sort.iou_matching import iou
from application_util import visualization
from matplotlib import pyplot as plt
from tqdm import tqdm
from PIL import Image

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import normalize
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression

def gather_sequence_info(video_name, video_path, feat_path):
    detections = np.load(feat_path)

    cap = cv2.VideoCapture(video_path)
    ret, frame = cap.read()
    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))-1
    cap.release()

    image_size = np.array(frame).shape[:-1] 
    # print(image_size)

#     min_frame_idx = int(detections[:, 0].min())
    min_frame_idx = int(1)
    max_frame_idx = int(length)

    feature_dim = detections.shape[1] - 10 if detections is not None else 0
    seq_info = {
        "sequence_name": video_name,
        "detections": detections,
        "image_size": image_size,
        "min_frame_idx": min_frame_idx,
        "max_frame_idx": max_frame_idx,
        "feature_dim": feature_dim,
        "update_ms": None
    }
    return seq_info

# ========================================================================================================


def capture(video_path, cap_dir, results, seq_info, is_plot=False):

    if os.path.exists(cap_dir):
        shutil.rmtree(cap_dir)
    os.makedirs(cap_dir)

    cap = cv2.VideoCapture(video_path)

    N_track = int(max(results[:,1]))
    subplot_x = 6
    subplot_y = int(math.ceil(N_track/subplot_x))
    print('Total Tracks:', N_track)
    print('Subplot', subplot_y, subplot_x)

    image_size = seq_info['image_size']
    points = {}
    captured = []

    with tf.Session() as sess:
        for frame_idx in tqdm(range(
                            seq_info['min_frame_idx'], 
                            seq_info['max_frame_idx'] + 1), 'capturing output'):
        
            image_np = np.array(cap.read()[1])

            mask = results[:, 0].astype(np.int) == frame_idx
            track_ids = results[mask, 1].astype(np.int)
            boxes = results[mask, 2:6]

            for track_id, box in zip(track_ids, boxes):
                if(track_id not in captured):
                    captured.append(track_id)

                    l,t,w,h = np.array(box).astype(int)
                    if(l<0): l=0 # if xmin is negative 
                    if(t<0): t=0 # if ymin is negative

                    if(l+w > image_size[1]): w=image_size[1]-l # if xmax exceeds width
                    if(t+h > image_size[0]): h=image_size[0]-t # if ymax exceeds height

                    cropped_image = sess.run(tf.image.crop_to_bounding_box(image_np, t, l, h, w))
                    cropped_image = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB)

                    img = Image.fromarray(cropped_image)
                    img.save(os.path.join(cap_dir, str(track_id)+'.jpg'))

                    if(is_plot):
                        plt.subplot(subplot_y, subplot_x, len(captured))
                        plt.imshow(cropped_image)
                        plt.title(str(track_id)+', '+str(frame_idx))

    cap.release()

    if(is_plot):
        plt.subplots_adjust(top=0.92, bottom=0.08, left=0.1, right=0.95, hspace=0.5, wspace=0.8)
        plt.show()

# ========================================================================================================
   
def savitzky_golay(y, window_size, order, deriv=0, rate=1):
    window_size = np.abs(np.int(window_size))
    order = np.abs(np.int(order))
   
    if window_size % 2 != 1 or window_size < 1:
        raise TypeError("window_size size must be a positive odd number")
    if window_size < order + 2: # order should be less than or equal window-2
        raise TypeError("window_size is too small for the polynomials order")
        
    order_range = range(order+1)
    half_window = (window_size -1) // 2
    
    # precompute coefficients
    b = np.mat([[k**i for i in order_range] for k in range(-half_window, half_window+1)])
    m = np.linalg.pinv(b).A[deriv] * rate**deriv * factorial(deriv)
    
    # pad the signal at the extremes with
    # values taken from the signal itself
    firstvals = y[0] - np.abs( y[1:half_window+1][::-1] - y[0] )
    lastvals = y[-1] + np.abs(y[-half_window-1:-1][::-1] - y[-1])
    y = np.concatenate((firstvals, y, lastvals))
    
    return np.convolve( m[::-1], y, mode='valid')

# ========================================================================================================

def golay_filter(df_track, window_size=45, order=5):
    if(len(df_track) <= window_size):
        return df_track
    df_track[2] = savitzky_golay(df_track[2].values, window_size=window_size, order=order, deriv=0, rate=1)
    df_track[3] = savitzky_golay(df_track[3].values, window_size=window_size, order=order, deriv=0, rate=1)
    df_track[4] = savitzky_golay(df_track[4].values, window_size=window_size, order=order, deriv=0, rate=1)
    df_track[5] = savitzky_golay(df_track[5].values, window_size=window_size, order=order, deriv=0, rate=1)
    return df_track
    
def poly_interpolate(df_track):
    model = make_pipeline(PolynomialFeatures(5), Ridge(solver='svd'))
    X = np.array(df_track.index).reshape(-1, 1)
    df_track[2] = model.fit(X, df_track[2]).predict(X)
    df_track[3] = model.fit(X, df_track[3]).predict(X)
    df_track[4] = model.fit(X, df_track[4]).predict(X)
    df_track[5] = model.fit(X, df_track[5]).predict(X)
    return df_track

def moving_avg(df_track, window=5):
    df_haed = df_track[[2,3,4,5]][:window-1]
    df_tail = df_track[[2,3,4,5]].rolling(window=window).mean()[window-1:]
    df_track[[2,3,4,5]] = pd.concat([df_haed, df_tail], axis=0)
    return df_track

def smooth(df, smooth_method):
    polynomials = []
    
    From, To = min(df[1]), max(df[1])+1
    for track_id in range(From, To):
        df_track = df.loc[df[1]==track_id].copy()

        if(smooth_method == 'poly'): df_track = poly_interpolate(df_track)
        elif(smooth_method == 'moving'): df_track = moving_avg(df_track)
        elif(smooth_method == 'golay'): df_track = golay_filter(df_track)
            
        polynomials.append(df_track)
#     do_range = df[1].unique().astype(str)
#     for track_id in do_range:
#         df_track = df.loc[df[1]==track_id].copy()

#         if(smooth_method == 'poly'): df_track = poly_interpolate(df_track)
#         elif(smooth_method == 'moving'): df_track = moving_avg(df_track)
#         elif(smooth_method == 'golay'): df_track = golay_filter(df_track)
            
#         polynomials.append(df_track)

    df_smooth = pd.concat(polynomials)
    df_smooth = df_smooth.sort_index()
    return df_smooth.values

# ========================================================================================================

def run(video_path, track_path, feat_path, save_output, out_dir, cap_dir, concat, smoothing, save_fig, is_plot, lag = 30):
    video_name = os.path.basename(video_path)
    seq_info = gather_sequence_info(video_name, video_path, feat_path)
   
    if(concat): track_path = track_path[:-4]+'_join.csv'

    df = pd.read_csv(track_path, header=None)

    if(smoothing): results = smooth(df, smooth_method='golay')
    else: results = df.values

    if(save_fig):
        capture(video_path, cap_dir, results, seq_info, is_plot=is_plot)
        return


    cap = cv2.VideoCapture(video_path)
    print('Video Path:', video_path,'\tFeatures:', feat_path)

    print(lag)
    points = {}

    def frame_callback(vis, frame_idx):
        print("Frame idx", frame_idx)
        image_np = np.array(cap.read()[1])
        vis.set_image(image_np)

        mask = results[:, 0].astype(np.int) == frame_idx
        track_ids = results[mask, 1].astype(np.str)
        boxes = results[mask, 2:6]
#         F_boxes = results[mask, 10:14]
        
#         print(F_boxes.shape)
#         if len(F_boxes)>0 :
#             for i ,f_boxe  in enumerate(F_boxes):
#                 F_boxes[i][0] = F_boxes[i][0]+boxes[i][0]
#                 F_boxes[i][1] = F_boxes[i][1]+boxes[i][1]
        
        points[frame_idx] = []
        for track_id, box in zip(track_ids, boxes):
            l,t,w,h = np.array(box).astype(int)
            x, y = int(l+w/2), int(t+h)
            points[frame_idx].append([track_id, x, y])
        
        if(frame_idx > lag):
            remove_idx = frame_idx-lag
            if remove_idx in points:
                del points[remove_idx]
        
        vis.draw_groundtruth(track_ids, boxes, points)

    visualizer = visualization.Visualization(seq_info, update_ms=50)

    if save_output:
        if(concat):
            visualizer.viewer.enable_videowriter(os.path.join(out_dir, video_name[:-4]+'_opt.avi'))
        else:
            visualizer.viewer.enable_videowriter(os.path.join(out_dir, video_name[:-4]+'_reg.avi'))

    visualizer.run(frame_callback)

#     cap.release()
    
#     cv2.destroyAllWindows()


# ========================================================================================================





c:\users\kengamd\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\kengamd\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\kengamd\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\keng

In [ ]:
l1 = [
    '120_20200306120830.avi',
 '120_20200306120845.avi',
 '120_20200306120900.avi',
 '120_20200306120915.avi']
 

In [ ]:
l1 = [
#     '220_20200306113215.avi',
 '220_20200306113230.avi',
 '220_20200306113245.avi']

In [ ]:
l1 = ['120_20200306113215.avi',
 '120_20200306113230.avi',
 '120_20200306113245.avi']

In [ ]:
l1 = [
#     '121_20200306113215.avi',
 '121_20200306113230.avi',
 '121_20200306113245.avi']

In [ ]:
l1 = ['121_20200306120830.avi',
 '121_20200306120845.avi',
 '121_20200306120900.avi',
 '121_20200306120915.avi']

In [ ]:
l1 = ['121_20200422144115.avi'] 
l2 = ['120_20200422144230.avi']
# l1 = ['220_20200422144230.avi','220_20200422144245.avi']
l1 = ['122_20200422144230.avi','122_20200422144245.avi']

In [3]:
videos = os.listdir('./dataset/videos')

In [4]:
videos

['120_20200422144000.avi',
 '120_20200422144015.avi',
 '120_20200422144030.avi',
 '120_20200422144045.avi',
 '120_20200422144100.avi',
 '120_20200422144115.avi',
 '120_20200422144130.avi',
 '120_20200422144145.avi',
 '120_20200422144200.avi',
 '120_20200422144215.avi',
 '120_20200422144230.avi',
 '120_20200422144245.avi',
 '120_20200422144300.avi',
 '120_20200422144315.avi',
 '120_20200422144330.avi',
 '120_20200422144345.avi',
 '120_20200422144400.avi',
 '120_20200422144415.avi',
 '120_20200422144430.avi',
 '120_20200422144445.avi',
 '120_20200422144500.avi',
 '120_20200422144515.avi',
 '120_20200422144530.avi',
 '120_20200422144545.avi',
 '120_20200422144600.avi',
 '120_20200422144615.avi',
 '120_20200422144630.avi',
 '120_20200422144645.avi',
 '120_20200422144700.avi',
 '120_20200422144715.avi',
 '120_20200422144730.avi',
 '120_20200422144745.avi',
 '120_20200422144800.avi',
 '120_20200422144815.avi',
 '120_20200422144830.avi',
 '120_20200422144845.avi',
 '120_20200422144900.avi',
 

In [8]:
l1=[]
for n in videos :
    if n.startswith("120") :
        l1.append(n)

In [9]:
len(l1)

76

In [10]:
# test_video = 'front-509-20200203143657.avi'
# test_video = 'Door-1.mp4'

video_dir = "dataset/videos"

feat_dir = "dataset/features"

tracks_dir= "dataset/tracks"

save_output = True

out_dir = 'dataset/outputs'

cap_dir = 'dataset/captures'

concat = False

smoothing = False

save_fig = False

is_plot = False

videos = os.listdir(video_dir)
videos.sort()
for test_video in l1 : 
    for video_name in videos:
        if(video_name != test_video and test_video != '' ): 
            continue
        try:
            run(video_path = os.path.join(video_dir, video_name), 
                track_path = os.path.join(tracks_dir, video_name[:-3]+'csv'),
                feat_path  = os.path.join(feat_dir, video_name[:-3]+'npy'), 
                save_output = save_output, # save tracking video 
                out_dir = out_dir,
                cap_dir = os.path.join(cap_dir, video_name[:-4]),
                concat = concat,
                smoothing = smoothing,
                save_fig = save_fig, # save captured human
                is_plot = is_plot) # subplot of captured human

        except FileNotFoundError:
            print(video_name + " has not yet been generated.")
        

Video Path: dataset/videos\220_20200422144000.avi 	Features: dataset/features\220_20200422144000.npy
30


  0%|                                                                                          | 0/292 [00:00<?, ?it/s]

Frame idx 1
Frame idx 2
Frame idx 3


  1%|▊                                                                                 | 3/292 [00:00<00:12, 22.83it/s]

Frame idx 4
Frame idx 5


...

Frame idx 280
Frame idx 281


 97%|█████████████████████████████████████████████████████████████████████████████▎  | 281/291 [00:14<00:00, 19.32it/s]

Frame idx 282
Frame idx 283


 97%|█████████████████████████████████████████████████████████████████████████████▊  | 283/291 [00:14<00:00, 19.18it/s]

KeyboardInterrupt: 

In [ ]:
# test_video = 'Front-HW-20200203143657.avi'

video_dir = "../dataset/videos"

feat_dir = "../dataset/features"

tracks_dir= "../dataset/tracks"

save_output = True

out_dir = '../dataset/outputs'

cap_dir = '../dataset/captures'

concat = False

smoothing = False

save_fig = False

is_plot = False

def run_showresult(video_name):
    videos = os.listdir(video_dir)
    videos.sort()
    
    if(video_name in videos): 
        run(video_path = os.path.join(video_dir, video_name), 
            track_path = os.path.join(tracks_dir, video_name[:-3]+'csv'),
            feat_path  = os.path.join(feat_dir, video_name[:-3]+'npy'), 
            save_output = save_output, # save tracking video 
            out_dir = out_dir,
            cap_dir = os.path.join(cap_dir, video_name[:-4]),
            concat = concat,
            smoothing = smoothing,
            save_fig = save_fig, # save captured human
            is_plot = is_plot) # subplot of captured human
#         print('000000000000')

l1 = [
    '121_20200306120830.avi',
 '121_20200306120845.avi',
 '121_20200306120900.avi',
 '121_20200306120915.avi'] 

l2 = [
    '120_20200306120830.avi',
 '120_20200306120845.avi',
 '120_20200306120900.avi',
 '120_20200306120915.avi']
 
l3 = [
    '220_20200306120830.avi',
 '220_20200306120845.avi',
 '220_20200306120900.avi',
 '220_20200306120915.avi']   
# l1 = ['front-509-20200203143657.avi','front-509-20200203143713.avi'] 
# l2 = ['Front-HW-20200203143657.avi','Front-HW-20200203143713.avi']

import threading 

for l1_v,l2_v,l3_v in zip(l1,l2,l3):
    print((l1_v,l2_v,l3_v))
    t1 = threading.Thread(target=run_showresult, args=(l1_v,)) 
    t2 = threading.Thread(target=run_showresult, args=(l2_v,)) 
    t3 = threading.Thread(target=run_showresult, args=(l3_v,))

    # starting thread 1 
    t1.start() 
    # starting thread 2 
    t2.start() 
    # starting thread 2 
    t3.start() 
    
    # wait until thread 1 is completely executed 
    t1.join() 
    # wait until thread 2 is completely executed 
    t2.join()
    # wait until thread 2 is completely executed 
    t3.join()
    
    

# both threads completely executed 
print("Done!") 



In [34]:
d = {}

In [35]:
d['1_0'] = '20'
d['1_5'] = '25'

In [36]:
d

{'1_0': '20', '1_5': '25'}

In [38]:
pd.DataFrame.from_dict(d, orient='index').to_csv('fff.csv')